# 05 Tool
- https://docs.langchain.com/oss/python/langchain/tools
- Tool이란?
    - 도구는 에이전트가 **행동을 수행**하기 위해 호출하는 구성 요소
    - **명확하게 정의된 입력과 출력**을 통해 세계와 상호작용할 수 있도록 하여 모델 역량을 확장
    - 도구는 **호출 가능한 함수**와 그 **입력 스키마**를 캡슐화합니다.
    - 이 값들은 호환 가능한 채팅 모델에 전달되어, 모델이 **도구를 호출할지 여부**와 **어떤 인수**로 사용할지 결정할 수 있습니다.
    - 이러한 상황에서 툴 호출은 모델이 지정된 **입력 스키마에 부합하는 요청을 생성**할 수 있게 합니다.

## Tool Definition

- 간단한 도구 정의
    - `@tool` 을 통한 도구 정의
    - 기본적으로 함수의 docstring은 도구가 설명되어 모델이 언제 사용해야 하는지 이해할 수 있도록 돕습니다
    - 타입 힌트는 도구의 입력 스키마를 정의하기 때문에 필요합니다. 모델이 도구의 목적을 이해할 수 있도록 간결해야 합니다.

In [21]:
from langchain.tools import tool

@tool
def search_database(query: str, limit: int = 10) -> str:
    """Search the customer database for records matching the query.

    Args:
        query: Search terms to look for
        limit: Maximum number of results to return
    """
    return f"Found {limit} results for '{query}'"

In [22]:
from rich.pretty import pprint as rpprint
rpprint(search_database)

StructuredTool(
│   name='search_database',
│   description='Search the customer database for records matching the query.\n\nArgs:\n    query: Search terms to look for\n    limit: Maximum number of results to return',
│   args_schema=<class 'langchain_core.utils.pydantic.search_database'>,
│   func=<function search_database at 0x7829d4d63240>
)

- 사용자 지정 이름

In [23]:
@tool("web_search")  # Custom name - 오버라이드
def search(query: str) -> str:
    """Search the web for information."""
    return f"Results for: {query}"

print(search.name)  # web_search

web_search


In [24]:
rpprint(search)

StructuredTool(
│   name='web_search',
│   description='Search the web for information.',
│   args_schema=<class 'langchain_core.utils.pydantic.web_search'>,
│   func=<function search at 0x7829d4d62520>
)

- 사용자 지정 도구 설명

In [25]:
@tool("calculator", description="Performs arithmetic calculations. Use this for any math problems.")
def calc(expression: str) -> str:
    """Evaluate mathematical expressions."""
    return str(eval(expression))

In [26]:
rpprint(calc)

StructuredTool(
│   name='calculator',
│   description='Performs arithmetic calculations. Use this for any math problems.',
│   args_schema=<class 'langchain_core.utils.pydantic.calculator'>,
│   func=<function calc at 0x7829d4d63880>
)

- Pydantic을 활용한 도구 정의

In [27]:
from pydantic import BaseModel, Field
from typing import Literal

class WeatherInput(BaseModel):
    """Input for weather queries."""
    location: str = Field(description="City name or coordinates")
    units: Literal["celsius", "fahrenheit"] = Field(
        default="celsius",
        description="Temperature unit preference"
    )
    include_forecast: bool = Field(
        default=False,
        description="Include 5-day forecast"
    )

@tool(args_schema=WeatherInput)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

In [28]:
rpprint(get_weather)

StructuredTool(
│   name='get_weather',
│   description='Get current weather and optional forecast.',
│   args_schema=<class '__main__.WeatherInput'>,
│   func=<function get_weather at 0x7829d4d634c0>
)

- Json 스키마를 활용한 도구 정의

In [29]:
weather_schema = {
    "type": "object",
    "properties": {
        "location": {"type": "string"},
        "units": {"type": "string"},
        "include_forecast": {"type": "boolean"}
    },
    "required": ["location", "units", "include_forecast"]
}

@tool(args_schema=weather_schema)
def get_weather(location: str, units: str = "celsius", include_forecast: bool = False) -> str:
    """Get current weather and optional forecast."""
    temp = 22 if units == "celsius" else 72
    result = f"Current weather in {location}: {temp} degrees {units[0].upper()}"
    if include_forecast:
        result += "\nNext 5 days: Sunny"
    return result

In [30]:
rpprint(get_weather)

StructuredTool(
│   name='get_weather',
│   description='Get current weather and optional forecast.',
│   args_schema={
│   │   'type': 'object',
│   │   'properties': {
│   │   │   'location': {'type': 'string'},
│   │   │   'units': {'type': 'string'},
│   │   │   'include_forecast': {'type': 'boolean'}
│   │   },
│   │   'required': ['location', 'units', 'include_forecast']
│   },
│   func=<function get_weather at 0x7829d4de8860>
)

## Tool Runtime

- Tool Runtime이란?
    - State: 실행을 거치는 변경 가능한 데이터(e.g., messages, counters, custom fields)
    - Context: 사용자 ID, 세션 세부 정보, 애플리케이션 특화 구성 등 변경 불가능한 구성
    - Store: 대화 전반에 걸친 지속적인 장기 기억
    - Stream Writer: 도구가 실행될 때 custom updates를 스트리밍합니다
    - Config: 실행을 위한 `RunnableConfig`
    - Tool Call ID: 현재 도구의 고유 ID

![Tool Runtime](../assets/Tool_Runtime.png)

- Accessing state
    - Chat Model에는 ToolRuntime 매개변수가 보이지 않습니다.

In [31]:
from langchain.tools import tool, ToolRuntime

# Access the current conversation state
@tool
def summarize_conversation(
    runtime: ToolRuntime # ToolRuntime 추가
) -> str:
    """Summarize the conversation so far."""
    messages = runtime.state["messages"]

    human_msgs = sum(1 for m in messages if m.__class__.__name__ == "HumanMessage")
    ai_msgs = sum(1 for m in messages if m.__class__.__name__ == "AIMessage")
    tool_msgs = sum(1 for m in messages if m.__class__.__name__ == "ToolMessage")

    return f"Conversation has {human_msgs} user messages, {ai_msgs} AI responses, and {tool_msgs} tool results"

# Access custom state fields
@tool
def get_user_preference(
    pref_name: str,
    runtime: ToolRuntime  # ToolRuntime parameter is not visible to the model!!
) -> str:
    """Get a user preference value."""
    preferences = runtime.state.get("user_preferences", {})
    return preferences.get(pref_name, "Not set")

- Updating state
    - Command를 사용하여 Update를 처리 가능합니다.

In [32]:
from langgraph.types import Command
from langchain.messages import RemoveMessage
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.tools import tool, ToolRuntime

# Update the conversation history by removing all messages
@tool
def clear_conversation() -> Command:
    """Clear the conversation history."""

    return Command(
        update={
            "messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)],
        }
    )

# Update the user_name in the agent state
@tool
def update_user_name(
    new_name: str,
    runtime: ToolRuntime
) -> Command:
    """Update the user's name."""
    return Command(update={"user_name": new_name})

- Context 접근
    - `runtime.context` 를 통해 사용자 ID, 세션 세부 정보, 애플리케이션별 구성 등 변경 불가능한 구성 및 맥락 데이터를 접근할 수 있습니다.

In [33]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from dataclasses import dataclass
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime


USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

@dataclass
class UserContext:
    user_id: str

@tool
def get_account_info(runtime: ToolRuntime) -> str:
    """Get the current user's account information."""
    user_id = runtime.context.user_id # 컨텍스트 가져옴.

    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"Account holder: {user['name']}\nType: {user['account_type']}\nBalance: ${user['balance']}"
    return "User not found"

model = ChatOpenAI(model="gpt-4o")
agent = create_agent(
    model,
    tools=[get_account_info],
    context_schema=UserContext,
    system_prompt="You are a financial assistant."
)

result = agent.invoke(
    {"messages": [{"role": "user", "content": "What's my current balance?"}]},
    context=UserContext(user_id="user123")
)

In [36]:
rpprint(result)

{
│   'messages': [
│   │   HumanMessage(
│   │   │   content="What's my current balance?",
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='df32de5a-bc42-4973-bdf2-664f65c4a8af'
│   │   ),
│   │   AIMessage(
│   │   │   content='',
│   │   │   additional_kwargs={'refusal': None},
│   │   │   response_metadata={
│   │   │   │   'token_usage': {
│   │   │   │   │   'completion_tokens': 11,
│   │   │   │   │   'prompt_tokens': 50,
│   │   │   │   │   'total_tokens': 61,
│   │   │   │   │   'completion_tokens_details': {
│   │   │   │   │   │   'accepted_prediction_tokens': 0,
│   │   │   │   │   │   'audio_tokens': 0,
│   │   │   │   │   │   'reasoning_tokens': 0,
│   │   │   │   │   │   'rejected_prediction_tokens': 0
│   │   │   │   │   },
│   │   │   │   │   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
│   │   │   │   },
│   │   │   │   'model_provider': 'openai',
│   │   │   │   'model_name': 'gpt-4o-2024-08-06',
│   │   │   │   'system_fingerprint': 'fp_cbf1785567',
│   │   │   │   'id': 'chatcmpl-CeK50YXi4ooywrdf6kOgqHzOaLleN',
│   │   │   │   'service_tier': 'default',
│   │   │   │   'finish_reason': 'tool_calls',
│   │   │   │   'logprobs': None
│   │   │   },
│   │   │   id='lc_run--58ced47d-b080-49d8-80a2-4ee7a4fa6b35-0',
│   │   │   tool_calls=[
│   │   │   │   {
│   │   │   │   │   'name': 'get_account_info',
│   │   │   │   │   'args': {},
│   │   │   │   │   'id': 'call_4YcR2EWnULv8bbq2JcCVhX1Q',
│   │   │   │   │   'type': 'tool_call'
│   │   │   │   }
│   │   │   ],
│   │   │   usage_metadata={
│   │   │   │   'input_tokens': 50,
│   │   │   │   'output_tokens': 11,
│   │   │   │   'total_tokens': 61,
│   │   │   │   'input_token_details': {'audio': 0, 'cache_read': 0},
│   │   │   │   'output_token_details': {'audio': 0, 'reasoning': 0}
│   │   │   }
│   │   ),
│   │   ToolMessage(
│   │   │   content='Account holder: Alice Johnson\nType: Premium\nBalance: $5000',
│   │   │   name='get_account_info',
│   │   │   id='7729f6ac-81ee-44fb-9a47-86387e71664c',
│   │   │   tool_call_id='call_4YcR2EWnULv8bbq2JcCVhX1Q'
│   │   ),
│   │   AIMessage(
│   │   │   content='Your current balance is $5,000.',
│   │   │   additional_kwargs={'refusal': None},
│   │   │   response_metadata={
│   │   │   │   'token_usage': {
│   │   │   │   │   'completion_tokens': 10,
│   │   │   │   │   'prompt_tokens': 85,
│   │   │   │   │   'total_tokens': 95,
│   │   │   │   │   'completion_tokens_details': {
│   │   │   │   │   │   'accepted_prediction_tokens': 0,
│   │   │   │   │   │   'audio_tokens': 0,
│   │   │   │   │   │   'reasoning_tokens': 0,
│   │   │   │   │   │   'rejected_prediction_tokens': 0
│   │   │   │   │   },
│   │   │   │   │   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
│   │   │   │   },
│   │   │   │   'model_provider': 'openai',
│   │   │   │   'model_name': 'gpt-4o-2024-08-06',
│   │   │   │   'system_fingerprint': 'fp_cbf1785567',
│   │   │   │   'id': 'chatcmpl-CeK51LL7Zkxz3y5Fk9mg3AlKiZdMe',
│   │   │   │   'service_tier': 'default',
│   │   │   │   'finish_reason': 'stop',
│   │   │   │   'logprobs': None
│   │   │   },
│   │   │   id='lc_run--160bb51e-ae1e-4f4b-b862-1d95497d7090-0',
│   │   │   usage_metadata={
│   │   │   │   'input_tokens': 85,
│   │   │   │   'output_tokens': 10,
│   │   │   │   'total_tokens': 95,
│   │   │   │   'input_token_details': {'audio': 0, 'cache_read': 0},
│   │   │   │   'output_token_details': {'audio': 0, 'reasoning': 0}
│   │   │   }
│   │   )
│   ]
}

- Memory (Store)
    - 스토어를 통해 대화 간 지속적인 데이터에 접근할 수 있습니다. 저장소는 runtime.store 을 통해 접근할 수 있으며, 사용자 전용 또는 애플리케이션별 데이터를 저장하고 검색할 수 있습니다.

In [38]:
from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime


# Access memory
@tool
def get_user_info(user_id: str, runtime: ToolRuntime) -> str:
    """Look up user info."""
    store = runtime.store
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "Unknown user"

# Update memory
@tool
def save_user_info(user_id: str, user_info: dict[str, Any], runtime: ToolRuntime) -> str:
    """Save user info."""
    store = runtime.store
    store.put(("users",), user_id, user_info)
    return "Successfully saved user info."

store = InMemoryStore()
agent = create_agent(
    model,
    tools=[get_user_info, save_user_info],
    store=store
)

# First session: save user info
first = agent.invoke({
    "messages": [{"role": "user", "content": "Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev"}]
})

# Second session: get user info
second = agent.invoke({
    "messages": [{"role": "user", "content": "Get user info for user with id 'abc123'"}]
})
# Here is the user info for user with ID "abc123":
# - Name: Foo
# - Age: 25
# - Email: foo@langchain.dev

In [39]:
rpprint(first)

{
│   'messages': [
│   │   HumanMessage(
│   │   │   content='Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev',
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='88d4ae41-366e-48ee-b07c-937d538bdef3'
│   │   ),
│   │   AIMessage(
│   │   │   content='',
│   │   │   additional_kwargs={'refusal': None},
│   │   │   response_metadata={
│   │   │   │   'token_usage': {
│   │   │   │   │   'completion_tokens': 17,
│   │   │   │   │   'prompt_tokens': 90,
│   │   │   │   │   'total_tokens': 107,
│   │   │   │   │   'completion_tokens_details': {
│   │   │   │   │   │   'accepted_prediction_tokens': 0,
│   │   │   │   │   │   'audio_tokens': 0,
│   │   │   │   │   │   'reasoning_tokens': 0,
│   │   │   │   │   │   'rejected_prediction_tokens': 0
│   │   │   │   │   },
│   │   │   │   │   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
│   │   │   │   },
│   │   │   │   'model_provider': 'openai',
│   │   │   │   'model_name': 'gpt-4o-2024-08-06',
│   │   │   │   'system_fingerprint': 'fp_cbf1785567',
│   │   │   │   'id': 'chatcmpl-CeK6JU82kVYaW6hGFghpMdYhyXySh',
│   │   │   │   'service_tier': 'default',
│   │   │   │   'finish_reason': 'tool_calls',
│   │   │   │   'logprobs': None
│   │   │   },
│   │   │   id='lc_run--256a5f4a-57cd-464b-b282-662cdf7260d6-0',
│   │   │   tool_calls=[
│   │   │   │   {
│   │   │   │   │   'name': 'save_user_info',
│   │   │   │   │   'args': {'user_id': 'abc123'},
│   │   │   │   │   'id': 'call_sHPV1jTYWSSzD6PqBbpjkuEK',
│   │   │   │   │   'type': 'tool_call'
│   │   │   │   }
│   │   │   ],
│   │   │   usage_metadata={
│   │   │   │   'input_tokens': 90,
│   │   │   │   'output_tokens': 17,
│   │   │   │   'total_tokens': 107,
│   │   │   │   'input_token_details': {'audio': 0, 'cache_read': 0},
│   │   │   │   'output_token_details': {'audio': 0, 'reasoning': 0}
│   │   │   }
│   │   ),
│   │   ToolMessage(
│   │   │   content="Error invoking tool 'save_user_info' with kwargs {'user_id': 'abc123', 'runtime': ToolRuntime(state={'messages': [HumanMessage(content='Save the following user: userid: abc123, name: Foo, age: 25, email: foo@langchain.dev', additional_kwargs={}, response_metadata={}, id='88d4ae41-366e-48ee-b07c-937d538bdef3'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 90, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-CeK6JU82kVYaW6hGFghpMdYhyXySh', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--256a5f4a-57cd-464b-b282-662cdf7260d6-0', tool_calls=[{'name': 'save_user_info', 'args': {'user_id': 'abc123'}, 'id': 'call_sHPV1jTYWSSzD6PqBbpjkuEK', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 'output_tokens': 17, 'total_tokens': 107, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}, context=None, config={'tags': [], 'metadata': {'langgraph_step': 2, 'langgraph_node': 'tools', 'langgraph_triggers': ('__pregel_push',), 'langgraph_path': ('__pregel_push', 0, False), 'langgraph_checkpoint_ns': 'tools:6fbb6cb2-a5fc-3557-7ae1-ad8d6759894e', 'checkpoint_ns': 'tools:6fbb6cb2-a5fc-3557-7ae1-ad8d6759894e'}, 'callbacks': <langchain_core.callbacks.manager.CallbackManager object at 0x7829d41d4850>, 'recursion_limit': 25, 'configurable': {'__pregel_runtime': Runtime(context=None, store=<langgraph.store.memory.InMemoryStore object at 0x7829d4101630>, stream_writer=<function Pregel.stream.<locals>.stream_writer at 0x7829e2aa7060>, previous=None), '__pregel_task_id': '6fbb6cb2-a5fc-3557-7ae1-ad8d6759894e', '__pregel_send'

In [40]:
rpprint(second)

{
│   'messages': [
│   │   HumanMessage(
│   │   │   content="Get user info for user with id 'abc123'",
│   │   │   additional_kwargs={},
│   │   │   response_metadata={},
│   │   │   id='645028bf-09a9-4618-ad15-e2ef9bc1ba7b'
│   │   ),
│   │   AIMessage(
│   │   │   content='',
│   │   │   additional_kwargs={'refusal': None},
│   │   │   response_metadata={
│   │   │   │   'token_usage': {
│   │   │   │   │   'completion_tokens': 17,
│   │   │   │   │   'prompt_tokens': 75,
│   │   │   │   │   'total_tokens': 92,
│   │   │   │   │   'completion_tokens_details': {
│   │   │   │   │   │   'accepted_prediction_tokens': 0,
│   │   │   │   │   │   'audio_tokens': 0,
│   │   │   │   │   │   'reasoning_tokens': 0,
│   │   │   │   │   │   'rejected_prediction_tokens': 0
│   │   │   │   │   },
│   │   │   │   │   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
│   │   │   │   },
│   │   │   │   'model_provider': 'openai',
│   │   │   │   'model_name': 'gpt-4o-2024-08-06',
│   │   │   │   'system_fingerprint': 'fp_cbf1785567',
│   │   │   │   'id': 'chatcmpl-CeK6QsBBeNHElNVWxekU72mYZTqrj',
│   │   │   │   'service_tier': 'default',
│   │   │   │   'finish_reason': 'tool_calls',
│   │   │   │   'logprobs': None
│   │   │   },
│   │   │   id='lc_run--f6c97f82-e6ea-450d-ab78-b8fc9106ffa2-0',
│   │   │   tool_calls=[
│   │   │   │   {
│   │   │   │   │   'name': 'get_user_info',
│   │   │   │   │   'args': {'user_id': 'abc123'},
│   │   │   │   │   'id': 'call_WWeElpzF1CEZNmU3LkYy94M9',
│   │   │   │   │   'type': 'tool_call'
│   │   │   │   }
│   │   │   ],
│   │   │   usage_metadata={
│   │   │   │   'input_tokens': 75,
│   │   │   │   'output_tokens': 17,
│   │   │   │   'total_tokens': 92,
│   │   │   │   'input_token_details': {'audio': 0, 'cache_read': 0},
│   │   │   │   'output_token_details': {'audio': 0, 'reasoning': 0}
│   │   │   }
│   │   ),
│   │   ToolMessage(
│   │   │   content='Unknown user',
│   │   │   name='get_user_info',
│   │   │   id='31d9c748-7a3a-4861-bf96-f3a4576b1381',
│   │   │   tool_call_id='call_WWeElpzF1CEZNmU3LkYy94M9'
│   │   ),
│   │   AIMessage(
│   │   │   content="It seems that there is no user information available for the user with the ID 'abc123'.",
│   │   │   additional_kwargs={'refusal': None},
│   │   │   response_metadata={
│   │   │   │   'token_usage': {
│   │   │   │   │   'completion_tokens': 20,
│   │   │   │   │   'prompt_tokens': 103,
│   │   │   │   │   'total_tokens': 123,
│   │   │   │   │   'completion_tokens_details': {
│   │   │   │   │   │   'accepted_prediction_tokens': 0,
│   │   │   │   │   │   'audio_tokens': 0,
│   │   │   │   │   │   'reasoning_tokens': 0,
│   │   │   │   │   │   'rejected_prediction_tokens': 0
│   │   │   │   │   },
│   │   │   │   │   'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
│   │   │   │   },
│   │   │   │   'model_provider': 'openai',
│   │   │   │   'model_name': 'gpt-4o-2024-08-06',
│   │   │   │   'system_fingerprint': 'fp_cbf1785567',
│   │   │   │   'id': 'chatcmpl-CeK6RO2oX2ohTKoHRJL8kXO3DAQMS',
│   │   │   │   'service_tier': 'default',
│   │   │   │   'finish_reason': 'stop',
│   │   │   │   'logprobs': None
│   │   │   },
│   │   │   id='lc_run--820aa14d-b3dc-401c-a263-2740d586ac00-0',
│   │   │   usage_metadata={
│   │   │   │   'input_tokens': 103,
│   │   │   │   'output_tokens': 20,
│   │   │   │   'total_tokens': 123,
│   │   │   │   'input_token_details': {'audio': 0, 'cache_read': 0},
│   │   │   │   'output_token_details': {'audio': 0, 'reasoning': 0}
│   │   │   }
│   │   )
│   ]
}

- Stream Writer

In [41]:
from langchain.tools import tool, ToolRuntime

@tool
def get_weather(city: str, runtime: ToolRuntime) -> str:
    """Get weather for a given city."""
    writer = runtime.stream_writer

    # Stream custom updates as the tool executes
    writer(f"Looking up data for city: {city}")
    writer(f"Acquired data for city: {city}")

    return f"It's always sunny in {city}!"

# Practice - 업데이트 예정

## Setup

In [ ]:
import uuid
import os
import operator
from typing import Literal, TypedDict, Annotated
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.types import Send
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# from langchain.tools import tool  # 이미 위에서 import됨
# from langchain_core.tools import ToolException  # 필요시 사용

## Define state schemas

In [ ]:
class OverallState(TypedDict):
    topic: str
    subjects: list[str]
    jokes: Annotated[list[str], operator.add]
    best_selected_joke: str

# Define Nodes, Edges

In [ ]:
def generate_topics(state: OverallState):
    return {"subjects": ["lions", "elephants", "penguins"]}

def generate_joke(state: OverallState):
    joke_map = {
        "lions": "Why don't lions like fast food? Because they can't catch it!",
        "elephants": "Why don't elephants use computers? They're afraid of the mouse!",
        "penguins": "Why don't penguins like talking to strangers at parties? Because they find it hard to break the ice."
    }
    return {"jokes": [joke_map[state["subject"]]]}

def continue_to_jokes(state: OverallState):
    """
    작업을 분할하여 각각에 대해 generate_joke를 실행
    Send API로 동적으로 병렬 실행 지시

    중요: 이 함수는 조건부 엣지 함수로 사용됩니다.
    """
    subjects = state["subjects"]
    print(f"[router] {len(subjects)}개 작업을 병렬 처리 시작")

    # 각 작업마다 worker Node 실행
    # Send(노드이름, 개별_상태_업데이트)
    return [Send("generate_joke", {"subject": s}) for s in state["subjects"]]

def best_joke(state: OverallState):
    return {"best_selected_joke": "penguins"}

# Build the graph

In [ ]:
# 그래프 생성
builder = StateGraph(OverallState)
builder.add_node("generate_topics", generate_topics)
builder.add_node("generate_joke", generate_joke)
builder.add_node("best_joke", best_joke)


# Edge 추가
builder.add_edge(START, "generate_topics")
# Send는 add_conditional_edges 없이 사용 불가능
builder.add_conditional_edges(
    "generate_topics",
    continue_to_jokes, 
    ["generate_joke"] # 동적 라우팅, asyncio.gather() 와 같이 노드 전부 실행 후 다음 노드로 감.
)

# NOTE: generate_joke들이 "모두 완료"되면 best_joke로 이동 (리듀서 패턴)
builder.add_edge("generate_joke", "best_joke")
builder.add_edge("best_joke", END)

# 컴파일
graph = builder.compile()

In [ ]:
# 그래프 그리기
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))

## Test

In [ ]:
import time
for step in graph.stream({"topic": "animals"}):
    print(step) # 한번에 하나씩 처리

---

## 병렬 실행 확인

In [ ]:
from datetime import datetime
import time

import asyncio

def generate_topics(state: OverallState):
    return {"subjects": ["lions", "elephants", "penguins"]}

async def generate_joke(state: OverallState):
    joke_map = {
        "lions": "Why don't lions like fast food? Because they can't catch it!",
        "elephants": "Why don't elephants use computers? They're afraid of the mouse!",
        "penguins": "Why don't penguins like talking to strangers at parties? Because they find it hard to break the ice."
    }

    print(f"🔄 {state["subject"]} 시작 ({5}초)")
    await asyncio.sleep(5)
    print(f"✅ {state["subject"]} 완료!")
    return {"jokes": [joke_map[state["subject"]]]}

def continue_to_jokes(state: OverallState):
    """
    작업을 분할하여 각각에 대해 generate_joke를 실행
    Send API로 동적으로 병렬 실행 지시

    중요: 이 함수는 조건부 엣지 함수로 사용됩니다.
    """
    subjects = state["subjects"]
    print(f"[router] {len(subjects)}개 작업을 병렬 처리 시작")

    # 각 작업마다 worker Node 실행
    # Send(노드이름, 개별_상태_업데이트)
    return [Send("generate_joke", {"subject": s}) for s in state["subjects"]]

async def best_joke(state: OverallState):
    print(f"🔄 best joke 선정 시작 ({5}초)")
    await asyncio.sleep(5)
    return {"best_selected_joke": "penguins"}

# 그래프 생성
builder = StateGraph(OverallState)
builder.add_node("generate_topics", generate_topics)
builder.add_node("generate_joke", generate_joke)
builder.add_node("best_joke", best_joke)


# Edge 추가
builder.add_edge(START, "generate_topics")
# Send는 add_conditional_edges 없이 사용 불가능
builder.add_conditional_edges(
    "generate_topics",
    continue_to_jokes, 
    ["generate_joke"] # 동적 라우팅
)

# NOTE: generate_joke들이 "모두 완료"되면 best_joke로 이동
builder.add_edge("generate_joke", "best_joke")
builder.add_edge("best_joke", END)

# 컴파일
graph = builder.compile()

In [ ]:
start = time.time()
async for step in graph.astream({"topic": "animals"}):
    elapsed = time.time() - start
    print(f"[{elapsed:.2f}초] {step}")

In [ ]:
start = time.time()
async for step in graph.astream({"topic": "animals"}, stream_mode="debug"):
    elapsed = time.time() - start
    print(f"[{elapsed:.2f}초] {step}")